In [1]:
import pandas as pd
import sys
from difflib import SequenceMatcher
from tqdm import tqdm
import time
from multiprocessing import Pool, cpu_count
import numpy as np

sys.path.append('../functions')
from hash_functions import (
    remove_accents
)

In [2]:
# Read official extracted docs 
official_matches = pd.read_csv('../../data/03_extracted/entities_extracted_official.csv')
official_matches = official_matches[~official_matches['art_id'].str.contains('_TRANS', na=False)]
official_matches.head()

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_2,Secretaría de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,La aplicación del presente Reglamento correspo...,8,30
1,6D0C4493,6D0C4493_2,Ley Nacional de Ejecución Penal,LEY_NACIONAL,FEDERAL_LAWS,"adscrito a los Centros Penitenciarios , Centro...",30,16
2,6D0C4493,6D0C4493_2,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,de Sanciones Administrativas y de Integración ...,30,5
3,6D0C4493,6D0C4493_3,Ley de Centros Penitenciarios de la Ciudad de ...,LEY_CDMX,CDMX_LAWS,"Para los efectos del presente Reglamento , ade...",14,30
4,6D0C4493,6D0C4493_3,Constitución Política de los Estados Unidos Me...,CONSTITUCION,FEDERAL_LAWS,"de sanciones penales , de reinserción psicosoc...",30,30


In [3]:
laws_official = official_matches[official_matches['pattern_group'].isin(['CDMX_LAWS','FEDERAL_LAWS'])]
laws_official['entity_text'] = laws_official['entity_text'].apply(remove_accents)

gov_official = official_matches[official_matches['pattern_group'].isin(['CDMX_OFFICIAL'])]
gov_official['entity_text'] = gov_official['entity_text'].apply(remove_accents)

/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_61863/3430584654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laws_official['entity_text'] = laws_official['entity_text'].apply(remove_accents)
/var/folders/3p/n363h71s3gg0h1brsrv7f_zr0000gn/T/ipykernel_61863/3430584654.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gov_official['entity_text'] = gov_official['entity_text'].apply(remove_accents)


In [4]:
print(laws_official.shape)
print(gov_official.shape)

(3581, 8)
(9615, 8)


In [5]:
# Read laws names
list_of_laws = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')

In [6]:
# Read list of government entities
list_of_entities = pd.read_csv('../../data/02_catalogs/entes-publicos_hash.csv')


In [7]:
def create_blocking_key(text, n_words=2):
    """
    Crea una clave de bloqueo basada en las primeras n palabras.
    """
    words = text.lower().split()
    return ' '.join(words[:min(n_words, len(words))])


def match_official_docs_blocking(oficial_docs_df, leyes_completas_df, threshold=0.8, n_words=2, id_column='doc_id', use_acronym=False):
    """
    Matching con blocking para reducir el número de comparaciones.
    Agrupa documentos por las primeras palabras antes de comparar.
    
    Args:
        oficial_docs_df: DataFrame con documentos oficiales
        leyes_completas_df: DataFrame con catálogo de leyes/entidades
        threshold: Umbral mínimo de similitud
        n_words: Número de palabras para crear bloques (default=2)
        id_column: Nombre de la columna ID a usar ('doc_id' o 'entity_id')
        use_acronym: Si True, primero busca match exacto en columna 'acronimo' antes de buscar en 'nombre'
    """
    start_time = time.time()
    
    print(f"Iniciando matching con BLOCKING")
    print(f"Documentos oficiales: {len(oficial_docs_df):,}")
    print(f"Referencias: {len(leyes_completas_df):,}")
    print(f"Threshold: {threshold}")
    print(f"Blocking: primeras {n_words} palabras")
    print(f"ID column: {id_column}")
    print(f"Usar acrónimos: {use_acronym}")
    
    # Si se usa acrónimos, crear un diccionario de match exacto para acrónimos
    acronym_dict = {}
    if use_acronym and 'acronimo' in leyes_completas_df.columns:
        for idx, row in leyes_completas_df.iterrows():
            if pd.notna(row['acronimo']) and row['acronimo'].strip():
                acronym_dict[row['acronimo'].strip().upper()] = row[id_column]
        print(f"Acrónimos cargados: {len(acronym_dict):,}")
    
    # Crear índice de bloques para el catálogo
    blocks = {}
    for idx, row in leyes_completas_df.iterrows():
        block_key = create_blocking_key(row['nombre'], n_words)
        if block_key not in blocks:
            blocks[block_key] = []
        blocks[block_key].append((row['nombre'], row[id_column]))
    
    print(f"Bloques creados: {len(blocks):,}")
    
    # Procesar cada documento oficial
    results = []
    total_comparisons = 0
    acronym_matches = 0
    name_matches = 0
    
    for idx, row in tqdm(oficial_docs_df.iterrows(), total=len(oficial_docs_df), desc="Procesando"):
        entity_text = row['entity_text']
        art_id = row['art_id']
        
        best_match = None
        best_score = 0
        match_type = None
        
        # Primero: intentar match exacto con acrónimos (si está habilitado)
        if use_acronym and acronym_dict:
            entity_upper = entity_text.strip().upper()
            if entity_upper in acronym_dict:
                best_match = acronym_dict[entity_upper]
                best_score = 1.0
                match_type = 'acronym'
                acronym_matches += 1
        
        # Si no se encontró por acrónimo, buscar por nombre con similitud
        if not best_match:
            # Obtener el bloque correspondiente
            block_key = create_blocking_key(entity_text, n_words)
            candidates = blocks.get(block_key, [])
            
            total_comparisons += len(candidates)
            
            # Buscar el mejor match solo dentro del bloque
            for law_name, matched_id in candidates:
                similarity = SequenceMatcher(None, entity_text.lower(), law_name.lower()).ratio()
                
                if similarity > best_score and similarity >= threshold:
                    best_score = similarity
                    best_match = matched_id
                    match_type = 'name'
            
            if best_match:
                name_matches += 1
        
        results.append({
            'art_id': art_id,
            'entity_text': entity_text,
            id_column: best_match if best_match else '',
            'similarity': best_score if best_match else 0,
            'match_type': match_type if match_type else ''
        })
    
    elapsed_time = time.time() - start_time
    avg_block_size = total_comparisons / len(oficial_docs_df) if len(oficial_docs_df) > 0 else 0
    reduction_factor = (len(oficial_docs_df) * len(leyes_completas_df)) / total_comparisons if total_comparisons > 0 else 0
    
    print(f"\nMatching completado en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
    print(f"Comparaciones realizadas: {total_comparisons:,}")
    print(f"Tamaño promedio de bloque: {avg_block_size:.1f}")
    print(f"Reducción de comparaciones: {reduction_factor:.1f}x")
    print(f"Matches encontrados: {sum(1 for r in results if r[id_column])}")
    if use_acronym:
        print(f"  - Por acrónimo: {acronym_matches}")
        print(f"  - Por nombre: {name_matches}")
    print(f"Velocidad: {len(oficial_docs_df)/elapsed_time:.1f} docs/segundo")
    
    return pd.DataFrame(results)


In [8]:
def process_chunk(args):
    """
    Función auxiliar para procesar un chunk de documentos en paralelo.
    """
    chunk_df, leyes_completas_list, threshold, id_column, acronym_dict = args
    results = []
    
    for _, row in chunk_df.iterrows():
        entity_text = row['entity_text']
        art_id = row['art_id']
        
        best_match = None
        best_score = 0
        match_type = None
        
        # Primero: intentar match exacto con acrónimos (si está disponible)
        if acronym_dict:
            entity_upper = entity_text.strip().upper()
            if entity_upper in acronym_dict:
                best_match = acronym_dict[entity_upper]
                best_score = 1.0
                match_type = 'acronym'
        
        # Si no se encontró por acrónimo, buscar por nombre
        if not best_match:
            # Buscar el mejor match
            for law_name, matched_id in leyes_completas_list:
                similarity = SequenceMatcher(None, entity_text.lower(), law_name.lower()).ratio()
                
                if similarity > best_score and similarity >= threshold:
                    best_score = similarity
                    best_match = matched_id
                    match_type = 'name'
        
        results.append({
            'art_id': art_id,
            'entity_text': entity_text,
            id_column: best_match if best_match else '',
            'similarity': best_score if best_match else 0,
            'match_type': match_type if match_type else ''
        })
    
    return results


def match_official_docs_parallel(oficial_docs_df, leyes_completas_df, threshold=0.8, n_jobs=None, id_column='doc_id', use_acronym=False):
    """
    Versión paralelizada del matching entre entity_text y name.
    Usa multiprocessing para acelerar el proceso.
    
    Args:
        oficial_docs_df: DataFrame con documentos oficiales
        leyes_completas_df: DataFrame con catálogo de leyes/entidades
        threshold: Umbral mínimo de similitud
        n_jobs: Número de procesos (None = usar todos los cores disponibles)
        id_column: Nombre de la columna ID a usar ('doc_id' o 'entity_id')
        use_acronym: Si True, primero busca match exacto en columna 'acronimo' antes de buscar en 'nombre'
    """
    start_time = time.time()
    
    # Determinar número de procesos
    if n_jobs is None:
        n_jobs = cpu_count()
    
    print(f"Iniciando matching PARALELO con {n_jobs} procesos")
    print(f"Documentos oficiales: {len(oficial_docs_df):,}")
    print(f"Referencias: {len(leyes_completas_df):,}")
    print(f"Threshold: {threshold}")
    print(f"ID column: {id_column}")
    print(f"Usar acrónimos: {use_acronym}")
    print(f"Total de comparaciones: {len(oficial_docs_df) * len(leyes_completas_df):,}")
    
    # Si se usa acrónimos, crear un diccionario de match exacto para acrónimos
    acronym_dict = {}
    if use_acronym and 'acronimo' in leyes_completas_df.columns:
        for idx, row in leyes_completas_df.iterrows():
            if pd.notna(row['acronimo']) and row['acronimo'].strip():
                acronym_dict[row['acronimo'].strip().upper()] = row[id_column]
        print(f"Acrónimos cargados: {len(acronym_dict):,}")
    
    # Convertir el catálogo a lista de tuplas para ser más eficiente
    leyes_completas_list = list(zip(
        leyes_completas_df['nombre'].values,
        leyes_completas_df[id_column].values
    ))
    
    # Dividir el DataFrame en chunks
    chunk_size = max(1, len(oficial_docs_df) // (n_jobs * 4))  # 4 chunks por proceso
    chunks = [oficial_docs_df.iloc[i:i+chunk_size] for i in range(0, len(oficial_docs_df), chunk_size)]
    
    print(f"Dividido en {len(chunks)} chunks de ~{chunk_size} documentos cada uno")
    
    # Preparar argumentos para cada chunk
    chunk_args = [(chunk, leyes_completas_list, threshold, id_column, acronym_dict if use_acronym else {}) for chunk in chunks]
    
    # Procesar en paralelo con barra de progreso
    all_results = []
    with Pool(processes=n_jobs) as pool:
        for result in tqdm(pool.imap(process_chunk, chunk_args), 
                          total=len(chunks), 
                          desc="Procesando chunks"):
            all_results.extend(result)
    
    elapsed_time = time.time() - start_time
    print(f"\nMatching completado en {elapsed_time:.2f} segundos ({elapsed_time/60:.2f} minutos)")
    print(f"Matches encontrados: {sum(1 for r in all_results if r[id_column])}")
    if use_acronym:
        acronym_matches = sum(1 for r in all_results if r.get('match_type') == 'acronym')
        name_matches = sum(1 for r in all_results if r.get('match_type') == 'name')
        print(f"  - Por acrónimo: {acronym_matches}")
        print(f"  - Por nombre: {name_matches}")
    print(f"Velocidad: {len(oficial_docs_df)/elapsed_time:.1f} docs/segundo")
    
    return pd.DataFrame(all_results)


In [9]:
# Ver cuántos cores tiene disponibles el sistema
print(f"Cores disponibles: {cpu_count()}")


Cores disponibles: 8


In [10]:
# Ejemplo de blocking: ver distribución de bloques
sample_blocks = {}
for _, row in list_of_laws.head(100).iterrows():
    key = create_blocking_key(row['nombre'], n_words=2)
    sample_blocks[key] = sample_blocks.get(key, 0) + 1

print(f"Ejemplo de bloques (muestra de 100 leyes):")
print(f"Bloques únicos: {len(sample_blocks)}")
print(f"Tamaño promedio: {sum(sample_blocks.values())/len(sample_blocks):.1f} leyes/bloque")
print(f"\nEjemplos de bloques:")
for key, count in list(sample_blocks.items())[:5]:
    print(f"  '{key}': {count} leyes")


Ejemplo de bloques (muestra de 100 leyes):
Bloques únicos: 10
Tamaño promedio: 10.0 leyes/bloque

Ejemplos de bloques:
  'constitucion politica': 1 leyes
  'ley de': 77 leyes
  'ley para': 6 leyes
  'ley organica': 5 leyes
  'ley del': 6 leyes


In [11]:
# Usar la versión con BLOCKING (la más rápida y eficiente)
results_laws = match_official_docs_blocking(laws_official, list_of_laws, threshold=0.9, n_words=2)

# Alternativas:
# results_laws = match_official_docs_parallel(laws_official, list_of_laws, threshold=0.9)  # Paralela sin blocking
# results_laws = match_official_docs(laws_official, list_of_laws, threshold=0.9)  # Secuencial

Iniciando matching con BLOCKING
Documentos oficiales: 3,581
Referencias: 783
Threshold: 0.9
Blocking: primeras 2 palabras
ID column: doc_id
Usar acrónimos: False
Bloques creados: 40


Procesando: 100%|██████████| 3581/3581 [00:28<00:00, 125.70it/s]


Matching completado en 28.52 segundos (0.48 minutos)
Comparaciones realizadas: 272,586
Tamaño promedio de bloque: 76.1
Reducción de comparaciones: 10.3x
Matches encontrados: 3581
Velocidad: 125.6 docs/segundo


In [12]:
results_laws['similarity'].describe()

count    3581.000000
mean        0.998771
std         0.004245
min         0.950000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: similarity, dtype: float64

In [13]:
results_laws[results_laws['similarity']<0.9]['entity_text'].unique()

array([], dtype=object)

In [14]:
# Esta bien, esta ley y reglamento no existen.

In [15]:
gov_official

,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_2,Secretaria de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,La aplicación del presente Reglamento correspo...,8,30
8,6D0C4493,6D0C4493_3,Secretaria de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,", bisexual , transgénero , transexual , traves...",30,30
10,6D0C4493,6D0C4493_3,Secretaria de Seguridad Ciudadana,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,Mayor de la Secretaría de Seguridad Ciudadana ...,30,13
13,6D0C4493,6D0C4493_16,Jefatura de Gobierno,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,II . Centro de Sanciones Administrativas y de ...,30,2
14,6D0C4493,6D0C4493_18,Jefatura de Gobierno,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,"acusadas , sentenciadas por resolución de auto...",30,30
...,...,...,...,...,...,...,...,...
13187,DC78E7E4,DC78E7E4_37,Secretaria de la Contraloria General,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,Son facultades del Contralor : Proponer a la *...,8,30
13188,DC78E7E4,DC78E7E4_37,Secretaria de la Contraloria General,CDMX_PODER_EJECUTIVO,CDMX_OFFICIAL,", obra pública , enajenación de bienes muebles...",30,30
13189,DC78E7E4,DC78E7E4_37,Auditoria Superior de la Ciudad de Mexico,CDMX_PODER_LEGISLATIVO,CDMX_OFFICIAL,temporal de los servidores públicos cuando a s...,30,30
13194,D73158BE,D73158BE_70,metro,CDMX_OTRO,CDMX_OFFICIAL,monitoreo del área provistos de tubería de 50 ...,30,30


In [16]:
list_of_entities

,unidad_responsable,nombre,gobierno_general,desc_gobierno_general,ambito,tipo_administracion,acronimo,name_nomalized,entity_id
0,01C001,JEFATURA DE GOBIERNO,1,PODER EJECUTIVO,CENTRAL,UNIDAD ADMINISTRATIVA,JG,JEFATURADEGOBIERNO,E2EEE4C1
1,01CD03,"CENTRO DE COMANDO, CONTROL, COMPUTO, COMUNICAC...",1,PODER EJECUTIVO,DESCONCENTRADO,ÓRGANO,C5,CENTRODECOMANDOCONTROLCOMPUTOCOMUNICACIONESYCO...,50B56830
2,01CD06,AGENCIA DIGITAL DE INNOVACION PUBLICA,1,PODER EJECUTIVO,DESCONCENTRADO,DEPENDENCIA,ADIP,AGENCIADIGITALDEINNOVACIONPUBLICA,11A7BB35
3,02C001,SECRETARIA DE GOBIERNO,2,PODER EJECUTIVO,CENTRAL,DEPENDENCIA,SEGOBCDMX,SECRETARIADEGOBIERNO,919DEC37
4,02CD01,ALCALDIA ALVARO OBREGON,2,PODER EJECUTIVO,GOBIERNO LOCAL,ALCALDÍA,AO,ALCALDIAALVAROOBREGON,FD7B16D9
...,...,...,...,...,...,...,...,...,...
107,09PDLR,CAJA DE PREVISION PARA TRABAJADORES A LISTA DE...,9,INSTITUCIONES PÚBLICAS DE SEGURIDAD SOCIAL,DESCENTRALIZADOS,ÓRGANOS,CAPTRALIR,CAJADEPREVISIONPARATRABAJADORESALISTADERAYA,D438D470
108,09PDPA,CAJA DE PREVISION DE LA POLICIA AUXILIAR,9,INSTITUCIONES PÚBLICAS DE SEGURIDAD SOCIAL,DESCENTRALIZADOS,ÓRGANOS,CAPREPA,CAJADEPREVISIONDELAPOLICIAAUXILIAR,12770D8F
109,09PDPP,CAJA DE PREVISION DE LA POLICIA PREVENTIVA,9,INSTITUCIONES PÚBLICAS DE SEGURIDAD SOCIAL,DESCENTRALIZADOS,ORGANISMOS,CAPREPOL,CAJADEPREVISIONDELAPOLICIAPREVENTIVA,5C6D544D
110,09PECM,"CORPORACION MEXICANA DE IMPRESION, S.A. DE C.V.",9,ENTIDADES PARAESTATALES NO FINANCIEROS,AUTÓNOMOS,ÓRGANOS,COMISA,CORPORACIONMEXICANADEIMPRESIONSADECV,B4576B9F


In [17]:
# Usar la versión con BLOCKING (la más rápida y eficiente)
# Para entidades gubernamentales, usamos use_acronym=True para hacer match exacto primero con acrónimos
results_gov = match_official_docs_blocking(gov_official, list_of_entities, threshold=0.9, n_words=2, id_column='entity_id', use_acronym=True)

# Alternativas:
# results_gov = match_official_docs_parallel(gov_official, list_of_entities, threshold=0.9)  # Paralela sin blocking
# results_gov = match_official_docs(gov_official, list_of_entities, threshold=0.9)  # Secuencial

Iniciando matching con BLOCKING
Documentos oficiales: 9,615
Referencias: 112
Threshold: 0.9
Blocking: primeras 2 palabras
ID column: entity_id
Usar acrónimos: True
Acrónimos cargados: 112
Bloques creados: 75


Procesando: 100%|██████████| 9615/9615 [00:02<00:00, 4621.11it/s]


Matching completado en 2.09 segundos (0.03 minutos)
Comparaciones realizadas: 55,771
Tamaño promedio de bloque: 5.8
Reducción de comparaciones: 19.3x
Matches encontrados: 9613
  - Por acrónimo: 491
  - Por nombre: 9122
Velocidad: 4608.6 docs/segundo


In [18]:
# Analizar resultados del matching
print("Distribución de tipos de match:")
print(results_gov['match_type'].value_counts())
print(f"\nTotal de matches: {len(results_gov[results_gov['entity_id'] != ''])}")
print(f"Sin match: {len(results_gov[results_gov['entity_id'] == ''])}")


Distribución de tipos de match:
match_type
name       9122
acronym     491
              2
Name: count, dtype: int64

Total de matches: 9613
Sin match: 2


In [19]:
# Ver ejemplos de cada tipo de match
print("Ejemplos de match por acrónimo:")
print(results_gov[results_gov['match_type'] == 'acronym'][['entity_text', 'entity_id', 'similarity']].head(10))
print("\n" + "="*80 + "\n")
print("Ejemplos de match por nombre:")
print(results_gov[results_gov['match_type'] == 'name'][['entity_text', 'entity_id', 'similarity']].head(10))


Ejemplos de match por acrónimo:
    entity_text entity_id  similarity
13          DIF  8663C012         1.0
270          C5  50B56830         1.0
320          C5  50B56830         1.0
321          C5  50B56830         1.0
336      SEDEMA  625ACAC7         1.0
422    bomberos  7CC77915         1.0
441          C5  50B56830         1.0
470          C5  50B56830         1.0
471    bomberos  7CC77915         1.0
555         DIF  8663C012         1.0


Ejemplos de match por nombre:
                               entity_text entity_id  similarity
0        Secretaria de Seguridad Ciudadana  6EA49C6C         1.0
1        Secretaria de Seguridad Ciudadana  6EA49C6C         1.0
2        Secretaria de Seguridad Ciudadana  6EA49C6C         1.0
3                     Jefatura de Gobierno  E2EEE4C1         1.0
4                     Jefatura de Gobierno  E2EEE4C1         1.0
5                Universidad de la Policia  5657AD8E         1.0
6                Universidad de la Policia  5657AD8E         1.

In [20]:
results_gov['similarity'].describe()

count    9615.000000
mean        0.999682
std         0.014556
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: similarity, dtype: float64

In [21]:
results_gov[results_gov['similarity']==0]['entity_text'].unique()

array(['Universidad Rosario Castellanos'], dtype=object)

In [22]:
results_gov['type_of_relation'] = 'mencion_entidad_gubernamental_oficial'

In [23]:
results_laws['type_of_relation'] = 'mencion_ley_oficial'

In [24]:
results_laws.to_csv('../../data/04_matched/oficial_docs_matched.csv', index=False)

In [25]:
results_gov.to_csv('../../data/04_matched/oficial_gov_matched.csv', index=False)